In [105]:
#Import the required packages
import urllib
import json
import numpy as np
import statsmodels.api as sm

In [10]:
mcg_url = "https://data.montgomerycountymd.gov/api/views/bedm-7sqa/rows.json?accessType=DOWNLOAD"#paste the website and store as mcg_url

#decode the json file and store in jupyter
mcg = urllib.request.urlopen(mcg_url)
json_string = mcg.read().decode('utf-8')
mcg_parsed_json1 = json.loads(json_string)
mcg_data = mcg_parsed_json1['data']
#check how many rows it contains in this json file
len(mcg_data)

9243

In [24]:
#use panda package to transform this json file into dataframe
import pandas as pd

mcgall = pd.DataFrame(mcg_data, columns=['1', '2', '3','4','5','6','7','8','9','10','11','12','13','14','15','16','17'])

In [38]:
#Since we only need those useful columns in our analysis, we keep those columns and store in mcg
mcg = mcgall[['9','10','11','12','13','14','15','16','17']]
mcg.columns = ['Generation', 'Age', 'Ethnic','Gender','Length_of_Service','Job_Class','Grade','Assignment_Category','Salary']

In [39]:
#check this dataframe
mcg

,Generation,Age,Ethnic,Gender,Length_of_Service,Job_Class,Grade,Assignment_Category,Salary
0,Generation X,42,White (Not Hispanic or Latino),Female,14,NON_MLS,25,Fulltime-Regular,90-99K
1,Generation X,44,Hispanic or Latino,Male,1,NON_MLS,15,Fulltime-Regular,40-49K
2,Generation X,41,Black or African American (Not Hispanic or Lat...,Female,11,NON_MLS,15,Fulltime-Regular,50-59K
3,Generation X,47,White (Not Hispanic or Latino),Male,12,NON_MLS,15,Fulltime-Regular,50-59K
4,Baby Boomers,59,Black or African American (Not Hispanic or Lat...,Female,2,NON_MLS,14,Fulltime-Regular,40-49K
...,...,...,...,...,...,...,...,...,...
9238,Millennial Generation,29,White (Not Hispanic or Latino),Male,5,NON_MLS,P4,Fulltime-Regular,60-69K
9239,Millennial Generation,29,White (Not Hispanic or Latino),Male,5,NON_MLS,P4,Fulltime-Regular,60-69K
9240,Millennial Generation,29,White (Not Hispanic or Latino),Male,5,NON_MLS,P4,Fulltime-Regular,60-69K
9241,Millennial Generation,29,White (Not Hispanic or Latino),Male,5,NON_MLS,P4,Fulltime-Regular,60-69K


In [40]:
#Because some of the columns only contains catagorical variables, we decided to create dummy variables for each catagorical variables
mcg = mcg.join(pd.get_dummies(mcg.Ethnic))
mcg = mcg.join(pd.get_dummies(mcg.Gender))
mcg = mcg.join(pd.get_dummies(mcg.Job_Class))
mcg = mcg.join(pd.get_dummies(mcg.Assignment_Category))
mcg = mcg.join(pd.get_dummies(mcg.Generation))

In [119]:
#check the forst ten rows to see if the dummy variables are successfully created
mcg.head(10)

,Generation,Age,Ethnic,Gender,Length_of_Service,Job_Class,Grade,Assignment_Category,Salary,American Indian or Alaska Native (Not Hispanic or Latino),...,MLS,NON_MLS,Fulltime-Regular,Parttime-Regular,Baby Boomers,Generation X,Millennial Generation,Post Millennials,Traditionalist/Silent Generation,Average_Salary
0,Generation X,42,White (Not Hispanic or Latino),Female,14,NON_MLS,25,Fulltime-Regular,90-99K,0,...,0,1,1,0,0,1,0,0,0,95000
1,Generation X,44,Hispanic or Latino,Male,1,NON_MLS,15,Fulltime-Regular,40-49K,0,...,0,1,1,0,0,1,0,0,0,45000
2,Generation X,41,Black or African American (Not Hispanic or Lat...,Female,11,NON_MLS,15,Fulltime-Regular,50-59K,0,...,0,1,1,0,0,1,0,0,0,55000
3,Generation X,47,White (Not Hispanic or Latino),Male,12,NON_MLS,15,Fulltime-Regular,50-59K,0,...,0,1,1,0,0,1,0,0,0,55000
4,Baby Boomers,59,Black or African American (Not Hispanic or Lat...,Female,2,NON_MLS,14,Fulltime-Regular,40-49K,0,...,0,1,1,0,1,0,0,0,0,45000
5,Baby Boomers,73,White (Not Hispanic or Latino),Male,16,NON_MLS,34,Fulltime-Regular,>=150K,0,...,0,1,1,0,1,0,0,0,0,150000
6,Baby Boomers,60,White (Not Hispanic or Latino),Male,26,NON_MLS,15,Fulltime-Regular,70-79K,0,...,0,1,1,0,1,0,0,0,0,75000
10,Generation X,40,Black or African American (Not Hispanic or Lat...,Female,14,NON_MLS,21,Fulltime-Regular,80-89K,0,...,0,1,1,0,0,1,0,0,0,85000
11,Baby Boomers,58,Black or African American (Not Hispanic or Lat...,Female,4,NON_MLS,15,Parttime-Regular,40-49K,0,...,0,1,0,1,1,0,0,0,0,45000
13,Baby Boomers,65,White (Not Hispanic or Latino),Female,13,NON_MLS,16,Fulltime-Regular,60-69K,0,...,0,1,1,0,1,0,0,0,0,65000


In [56]:
#Because the Grade column has many variables that contain non-numeric variables
#so we delete those variables and only use numerical variables.

mcg = mcg[mcg.Grade.apply(lambda x: x.isnumeric())]

In [57]:
#we use a loop to get the mean value of Salary, then store in a new column called Average_Salary
result = [] 
for value in mcg["Salary"]:
    if value == "<20K":
        result.append(20000)
    elif value == "20-29K":
        result.append(25000)
    elif value == "30-39K":
        result.append(35000)
    elif value == "40-49K":
        result.append(45000)
    elif value == "50-59K":
        result.append(55000)
    elif value == "60-69K":
        result.append(65000)
    elif value == "70-79K":
        result.append(75000)
    elif value == "80-89K":
        result.append(85000)
    elif value == "90-99K":
        result.append(95000)
    elif value == "100-109K":
        result.append(105000)
    elif value == "110-119K":
        result.append(115000)
    elif value == "120-129K":
        result.append(125000)
    elif value == "130-139K":
        result.append(135000)
    elif value == "140-149K":
        result.append(145000)
    else:
        result.append(150000)
     
        
mcg["Average_Salary"] = result    
mcg 

,Generation,Age,Ethnic,Gender,Length_of_Service,Job_Class,Grade,Assignment_Category,Salary,American Indian or Alaska Native (Not Hispanic or Latino),...,MLS,NON_MLS,Fulltime-Regular,Parttime-Regular,Baby Boomers,Generation X,Millennial Generation,Post Millennials,Traditionalist/Silent Generation,Average_Salary
0,Generation X,42,White (Not Hispanic or Latino),Female,14,NON_MLS,25,Fulltime-Regular,90-99K,0,...,0,1,1,0,0,1,0,0,0,95000
1,Generation X,44,Hispanic or Latino,Male,1,NON_MLS,15,Fulltime-Regular,40-49K,0,...,0,1,1,0,0,1,0,0,0,45000
2,Generation X,41,Black or African American (Not Hispanic or Lat...,Female,11,NON_MLS,15,Fulltime-Regular,50-59K,0,...,0,1,1,0,0,1,0,0,0,55000
3,Generation X,47,White (Not Hispanic or Latino),Male,12,NON_MLS,15,Fulltime-Regular,50-59K,0,...,0,1,1,0,0,1,0,0,0,55000
4,Baby Boomers,59,Black or African American (Not Hispanic or Lat...,Female,2,NON_MLS,14,Fulltime-Regular,40-49K,0,...,0,1,1,0,1,0,0,0,0,45000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9233,Generation X,48,Black or African American (Not Hispanic or Lat...,Male,4,NON_MLS,15,Fulltime-Regular,40-49K,0,...,0,1,1,0,0,1,0,0,0,45000
9234,Generation X,48,Black or African American (Not Hispanic or Lat...,Male,4,NON_MLS,15,Fulltime-Regular,40-49K,0,...,0,1,1,0,0,1,0,0,0,45000
9235,Generation X,48,Asian (Not Hispanic or Latino),Female,0,NON_MLS,24,Fulltime-Regular,70-79K,0,...,0,1,1,0,0,1,0,0,0,75000
9236,Generation X,48,Two or More Races (Not Hispanic or Latino),Male,6,NON_MLS,28,Fulltime-Regular,120-129K,0,...,0,1,1,0,0,1,0,0,0,125000


In [59]:
#we use all the variable in mcg except average salary as independent variable
#Since we already have all the dummy variable, so we delete those catagorical variables
independent=mcg.iloc[:, 1:28]
independent =independent.drop(labels='Ethnic',axis=1)
independent =independent.drop(labels='Gender',axis=1)
independent =independent.drop(labels='Job_Class',axis=1)
independent =independent.drop(labels='Assignment_Category',axis=1)
independent =independent.drop(labels='Salary',axis=1)
independent

,Age,Length_of_Service,Grade,American Indian or Alaska Native (Not Hispanic or Latino),Asian (Not Hispanic or Latino),Black or African American (Not Hispanic or Latino),Hispanic or Latino,Native Hawaiian/Other Pacific Islander(Not Hispanic/Latino),Two or More Races (Not Hispanic or Latino),Unreported,...,Male,MLS,NON_MLS,Fulltime-Regular,Parttime-Regular,Baby Boomers,Generation X,Millennial Generation,Post Millennials,Traditionalist/Silent Generation
0,42,14,25,0,0,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
1,44,1,15,0,0,0,1,0,0,0,...,1,0,1,1,0,0,1,0,0,0
2,41,11,15,0,0,1,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
3,47,12,15,0,0,0,0,0,0,0,...,1,0,1,1,0,0,1,0,0,0
4,59,2,14,0,0,1,0,0,0,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9233,48,4,15,0,0,1,0,0,0,0,...,1,0,1,1,0,0,1,0,0,0
9234,48,4,15,0,0,1,0,0,0,0,...,1,0,1,1,0,0,1,0,0,0
9235,48,0,24,0,1,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
9236,48,6,28,0,0,0,0,0,1,0,...,1,0,1,1,0,0,1,0,0,0


In [62]:
#use Average_Salary column as the dependent variable
Dependent = mcg['Average_Salary']
Dependent

0        95000
1        45000
2        55000
3        55000
4        45000
         ...  
9233     45000
9234     45000
9235     75000
9236    125000
9237     65000
Name: Average_Salary, Length: 5882, dtype: int64

In [ ]:
#ues liner model from the sklearn package to do the regression model
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(independent,Dependent)


In [103]:
#check the result of the liner model
model.score(independent,Dependent)

0.7094593259275835

In [131]:
#print out all the result and score of this model
r_sq = model.score(independent,Dependent)

print('coefficient of determination:', r_sq)

print('intercept:', model.intercept_)

print('coefficients:', model.coef_)

coefficient of determination: 0.7094593259275835
intercept: -14885.605307606325
coefficients: [ 1.19017663e+02  5.52944519e+02  3.74594457e+03 -5.71377788e+02
 -1.12176122e+03 -3.51521992e+03 -2.94743092e+03  1.17233533e+04
 -1.04424106e+03  7.34098546e+01 -2.59673227e+03 -8.86281128e+02
  8.86281128e+02 -1.13686838e-12  0.00000000e+00  7.73509678e+03
 -7.73509678e+03  2.41824802e+03  1.87340363e+03 -2.35148800e+03
  1.69561360e+03 -3.63577724e+03]
